In [ ]:
from pathlib import Path
import re
from shutil import copy

from lotr.file_utils import mirror_fish_folder
from tqdm import tqdm

In [ ]:
master_source = Path(r"\\FUNES\Shared\experiments\E0040_motions_cardinal")
dest_master_folder = Path(r"\\FUNES\Shared\experiments\E0071_lotr")

In [ ]:
all_folders_tomove = [f for f in master_source.glob("*/*[0-9]_f[0-9]*") if f.is_dir()]

In [ ]:


for f in tqdm(all_folders_tomove):
    fid = "".join(re.split(r"(\d+)", f.name.split("_")[1])[:2])  # take only actuall fish id of date and fish number
    date = f.name.split("_")[0]
    parent_dest_name = dest_master_folder / f"{date}_{fid}"
    parent_dest_name.mkdir(exist_ok=True)
    
    dest_name = parent_dest_name / f.name
    dest_name.mkdir(exist_ok=True)
    
    mirror_fish_folder(
            f, dest_name,
            file_patterns=[
                    "*behavior_log*",
                    "*stimulus_log*",
                    "*metadata.json",
                    "data_from_suite2p_unfiltered.h5",
                    "*video.mp4",
                    "*video_times.hdf5",
                    "*img.png",
                    "*select*"
                ]
        )

In [ ]:
for f in tqdm(all_folders_tomove):
    fid = "".join(re.split(r"(\d+)", f.name.split("_")[1])[:2])  # take only actuall fish id of date and fish number
    date = f.name.split("_")[0]
    parent_dest_name = dest_master_folder / f"{date}_{fid}"
    
    print(parent_dest_name)